In [129]:
import numpy as np;
import pandas as pd;
from  math import *;
from collections import Counter;

In [130]:
def out(fname, num):
    with open(fname, 'w') as f:
        f.write(str(num))

def out_many(fname, nums):
    with open(fname, 'w') as f:
        for num in nums:
            f.write(str(num)+' ')

In [166]:
train = pd.read_csv('coursera_sessions_train.txt', sep=';', names=['see', 'buy'])
test = pd.read_csv('coursera_sessions_test.txt', sep=';', names=['see', 'buy'])

In [167]:
train = train.fillna('')
test = test.fillna('')

In [168]:
train['see_items'] = train['see'].apply(lambda x: [int(ind) for ind in x.split(',')])
train['n_unique_see_items'] = train['see_items'].apply(lambda x: len(set(x)))
train['buy_items'] = train['buy'].apply(lambda x: [] if len(str(x))==0 else [int(float(ind)) for ind in str(x).split(',')])

test['see_items'] = test['see'].apply(lambda x: [int(ind) for ind in x.split(',')])
test['n_unique_see_items'] = test['see_items'].apply(lambda x: len(set(x)))
test['buy_items'] = test['buy'].apply(lambda x: [] if len(str(x))==0 else [int(float(ind)) for ind in str(x).split(',')])

In [169]:
see_count = Counter()
for s in train.see_items:
    see_count.update(s)
    
buy_count = Counter()
for b in train.buy_items:
    buy_count.update(b)

In [170]:
def sortByCounts(x, y):
    if (x[1] > y[1]):
        return 1
    elif (x[1] < y[1]):
        return -1
    return 0

def by_see_freq(k, ids):
    counts = [(id, see_count[id]) for id in pd.unique(ids)]
    counts.sort(cmp=sortByCounts, reverse=True)
    return [x[0] for x in counts[:k]]

def by_buy_freq(k, ids):
    counts = [(id, buy_count[id]) for id in pd.unique(ids)]
    counts.sort(cmp=sortByCounts, reverse=True)
    return [x[0] for x in counts[:k]]

def intersect(a, b):
    return [x for x in a if (x in b)]

In [174]:
sessions = train[train.apply(lambda x: len(x['buy_items']) > 0, axis=1)]
sessions.head(10)

,see,buy,see_items,n_unique_see_items,buy_items
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...",10,"[67, 60, 63]"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",10,[86]
19,"138,198,199,127",199,"[138, 198, 199, 127]",4,[199]
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",10,[303]
33,"352,353,352",352,"[352, 353, 352]",2,[352]
55,519,519,[519],1,[519]
64,"599,600,601,602","603,604,602,599,605,606,600","[599, 600, 601, 602]",4,"[603, 604, 602, 599, 605, 606, 600]"
72,"687,688,689,690,691,690,688,690,688,692","690,688","[687, 688, 689, 690, 691, 690, 688, 690, 688, ...",6,"[690, 688]"
89,"850,851,852",851,"[850, 851, 852]",3,[851]
93,"879,884,170,137,170,879,884,879,885,886,879,88...",879,"[879, 884, 170, 137, 170, 879, 884, 879, 885, ...",8,[879]


In [175]:
sessions.buy_items.values[0]

[67, 60, 63]

In [176]:
def avg_precision(data, k, recommend_model):
    precision = 0
    for _, row in data.iterrows():
        #n = min(k, row.n_unique_see_items)
        recommended = recommend_model(k, row.see_items)
        precision += len(intersect(recommended, row.buy_items)) / float(k)
    precision /= len(data)
    return precision
    
def avg_recall(data, k, recommend_model):
    recall = 0
    for _, row in data.iterrows():
        #n = min(k, row.n_unique_see_items)
        recommended = recommend_model(k, row.see_items)
        recall += len(intersect(recommended, row.buy_items)) / float(len(pd.unique(row.buy_items)))
    recall /= len(data)
    return recall

In [177]:
k = 1
avg_precision1_by_see = avg_precision(sessions, k, by_see_freq)
print 'Average precision for k=%i for By See Frequency algorythm: %f' % (k, avg_precision1_by_see)
avg_recall1_by_see = avg_recall(sessions, k, by_see_freq)
print 'Average recall for k=%i for By See Frequency algorythm: %f' % (k, avg_recall1_by_see)

Average precision for k=1 for By See Frequency algorythm: 0.512195
Average recall for k=1 for By See Frequency algorythm: 0.442634


In [178]:
k = 5
avg_precision5_by_see = avg_precision(sessions, k, by_see_freq)
print 'Average precision for k=%i for By See Frequency algorythm: %f' % (k, avg_precision5_by_see)
avg_recall5_by_see = avg_recall(sessions, k, by_see_freq)
print 'Average recall for k=%i for By See Frequency algorythm: %f' % (k, avg_recall5_by_see)

Average precision for k=5 for By See Frequency algorythm: 0.212528
Average recall for k=5 for By See Frequency algorythm: 0.824692


In [179]:
k = 1
avg_precision1_by_buy = avg_precision(sessions, k, by_buy_freq)
print 'Average precision for k=%i for By Buy Frequency algorythm: %f' % (k, avg_precision1_by_buy)
avg_recall1_by_buy = avg_recall(sessions, k, by_buy_freq)
print 'Average recall for k=%i for By Buy Frequency algorythm: %f' % (k, avg_recall1_by_buy)

k = 5
avg_precision5_by_buy = avg_precision(sessions, k, by_buy_freq)
print 'Average precision for k=%i for By Buy Frequency algorythm: %f' % (k, avg_precision5_by_buy)
avg_recall5_by_buy = avg_recall(sessions, k, by_buy_freq)
print 'Average recall for k=%i for By Buy Frequency algorythm: %f' % (k, avg_recall5_by_buy)

Average precision for k=1 for By Buy Frequency algorythm: 0.803769
Average recall for k=1 for By Buy Frequency algorythm: 0.688449
Average precision for k=5 for By Buy Frequency algorythm: 0.252550
Average recall for k=5 for By Buy Frequency algorythm: 0.926307


In [180]:
ans1 = [round(avg_recall1_by_see, 2), round(avg_precision1_by_see, 2), round(avg_recall5_by_see, 2), round(avg_precision5_by_see, 2)]
out_many('1.txt', ans1)
ans3 = [round(avg_recall1_by_buy, 2), round(avg_precision1_by_buy, 2), round(avg_recall5_by_buy, 2), round(avg_precision5_by_buy, 2)]
out_many('3.txt', ans3)

In [181]:
test_sessions = test[test.apply(lambda x: len(x['buy_items']) > 0, axis=1)]

,see,buy,see_items,n_unique_see_items,buy_items
7,"63,68,69,70,66,61,59,61,66,68","66,63","[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]",7,"[66, 63]"
14,"158,159,160,159,161,162",162,"[158, 159, 160, 159, 161, 162]",5,[162]
19,"200,201,202,203,204","201,205","[200, 201, 202, 203, 204]",5,"[201, 205]"
34,"371,372,371","371,373","[371, 372, 371]",2,"[371, 373]"
40,422,422,[422],1,[422]
47,"463,465,466,465,19,467,464,468,469,470,471,472...","462,460","[463, 465, 466, 465, 19, 467, 464, 468, 469, 4...",13,"[462, 460]"
57,"543,544,545,546",543,"[543, 544, 545, 546]",4,[543]
94,"900,894,904,894,902,904,905,906,895,894,901,89...",904,"[900, 894, 904, 894, 902, 904, 905, 906, 895, ...",21,[904]
115,1063,1063,[1063],1,[1063]
125,"1119,1120,1121,1122,1123,1124,1125,1126,1123,1...",1126,"[1119, 1120, 1121, 1122, 1123, 1124, 1125, 112...",8,[1126]


In [182]:
k = 1
avg_precision1_by_see_test = avg_precision(test_sessions, k, by_see_freq)
print 'Test: Average precision for k=%i for By See Frequency algorythm: %f' % (k, avg_precision1_by_see_test)
avg_recall1_by_see_test = avg_recall(test_sessions, k, by_see_freq)
print 'Test: Average recall for k=%i for By See Frequency algorythm: %f' % (k, avg_recall1_by_see_test)

k = 5
avg_precision5_by_see_test = avg_precision(test_sessions, k, by_see_freq)
print 'Test: Average precision for k=%i for By See Frequency algorythm: %f' % (k, avg_precision5_by_see_test)
avg_recall5_by_see_test = avg_recall(test_sessions, k, by_see_freq)
print 'Test: Average recall for k=%i for By See Frequency algorythm: %f' % (k, avg_recall5_by_see_test)

Test: Average precision for k=1 for By See Frequency algorythm: 0.481310
Test: Average recall for k=1 for By See Frequency algorythm: 0.417333
Test: Average precision for k=5 for By See Frequency algorythm: 0.203765
Test: Average recall for k=5 for By See Frequency algorythm: 0.800034


In [183]:
ans2 = [round(avg_recall1_by_see_test, 2), round(avg_precision1_by_see_test, 2), round(avg_recall5_by_see_test, 2), round(avg_precision5_by_see_test, 2)]
out_many('2.txt', ans2)

In [184]:
k = 1
avg_precision1_by_buy_test = avg_precision(test_sessions, k, by_buy_freq)
print 'Test: Average precision for k=%i for By buy Frequency algorythm: %f' % (k, avg_precision1_by_buy_test)
avg_recall1_by_buy_test = avg_recall(test_sessions, k, by_buy_freq)
print 'Test: Average recall for k=%i for By buy Frequency algorythm: %f' % (k, avg_recall1_by_buy_test)

k = 5
avg_precision5_by_buy_test = avg_precision(test_sessions, k, by_buy_freq)
print 'Test: Average precision for k=%i for By buy Frequency algorythm: %f' % (k, avg_precision5_by_buy_test)
avg_recall5_by_buy_test = avg_recall(test_sessions, k, by_buy_freq)
print 'Test: Average recall for k=%i for By buy Frequency algorythm: %f' % (k, avg_recall5_by_buy_test)

Test: Average precision for k=1 for By buy Frequency algorythm: 0.527694
Test: Average recall for k=1 for By buy Frequency algorythm: 0.460620
Test: Average precision for k=5 for By buy Frequency algorythm: 0.210095
Test: Average recall for k=5 for By buy Frequency algorythm: 0.820187


In [185]:
ans4 = [round(avg_recall1_by_buy_test, 2), round(avg_precision1_by_buy_test, 2), round(avg_recall5_by_buy_test, 2), round(avg_precision5_by_buy_test, 2)]
out_many('4.txt', ans4)